In [1]:
# Imports
import cv2
import os
from keras.models import load_model
import numpy as np
import time
from playsound import playsound

In [2]:
# Load and assign object detection algorithm files: face and eyepairs
face_recog = cv2.CascadeClassifier('haar cascade files\haarcascade_frontalface_alt.xml')
eye_pair = cv2.CascadeClassifier('haar cascade files\parojosG.xml')

# Assign labels for eye position
Position = ['Closed', 'Open']

# Assign variables
model = load_model('models/cnn3cat2.h5')
path = 'C:\\Users\\felix\\Desktop\\Final Project - Eye Alert\\Incidents'
cap = cv2.VideoCapture(0)
font = cv2.FONT_HERSHEY_COMPLEX_SMALL
color = (255, 255, 255)
color_warn = (0, 128, 255)
color_alert = (0, 0, 255)
count = 0
timer = 0
pred = []

In [3]:
# While loop to run during video capture
while(True):
    ret, frame = cap.read()
    height,width = frame.shape[:2]
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_recog.detectMultiScale(gray, minNeighbors = 5, scaleFactor = 1.1, minSize = (25, 25))
    eyes = eye_pair.detectMultiScale(gray)
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y) , (x + w, y + h) , (0, 255, 0) , 2)
    for (x, y, w, h) in eyes:
        roi_color = frame[y : y + h, x : x + w]
        eyepr = roi_color[y : y + h, x : x + w]
        cv2.rectangle(roi_color, (x, y),(x + w, y + h), (255, 0, 0), 2)
        count = count + 1
        eyepr = cv2.cvtColor(roi_color, cv2.COLOR_BGR2GRAY)
        eyepr = cv2.resize(eyepr, (24, 24))
        eyepr= eyepr / 255
        eyepr = eyepr.reshape(24, 24, -1)
        eyepr = np.expand_dims(eyepr, axis = 0)
        # Predict position of eyes
        pred = model.predict_classes(eyepr)
        if(pred[0] == 1):
            Position = 'Open'
        if(pred[0] == 0):
            Position = 'Closed'
        break
    # Set timer rules
    if(pred[0] == 0):
        timer = timer + 1
        cv2.putText(frame, "Eyes Closed", (10, height - 20), font, 1, color_alert, 1, cv2.LINE_AA)
    else:
        timer = timer - 1
        cv2.putText(frame, "Eyes Open", (10, height - 20), font, 1, color, 1, cv2.LINE_AA)
    if(timer < 0):
        timer = 0
    cv2.putText(frame, 'Net Total:' + str(timer), (460, height - 20), font, 1, color, 1, cv2.LINE_AA)
    # Set timer upper boundary, alert sound, and capture image of incident
    if(13 > timer > 9):
        cv2.rectangle(frame, (0, 0), (width, height), color_warn, thickness = 8)
    if(timer >= 13):
        cv2.rectangle(frame, (0, 0), (width, height), color_alert, thickness = 18)
    if(timer > 15):
        cv2.putText(frame, "Pull Over", (10, height - 42), font, 1, color_alert, 1, cv2.LINE_AA)
    if(timer > 17):
        cv2.imwrite(os.path.join(path, 'Incident Alert.jpg'), frame)
        playsound(r'C:\Users\felix\Desktop\Final Project - Eye Alert\buzzer.WAV')
    cv2.imshow('frame', frame)
    # Press q to escape the while loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
# Close capturing
cap.release()
cv2.destroyAllWindows()

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
